In [6]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import InputLayer

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

import pandas as pd

## Import the data

In [21]:
df = pd.read_csv('data.csv')

X = df.drop(columns = ['0']).copy()
y = df['0']

integer_encoded = LabelEncoder().fit_transform(y)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
y_onehot = OneHotEncoder(sparse=False).fit_transform(integer_encoded)


# Split data into testing, training and validation data
X_train, X_rem, y_train, y_rem = train_test_split(X, y_onehot, train_size=0.8)
X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5)

classes = ['a', 'b', 'c', 'd', 'e', 'f', 'g']
num_classes = len(classes)
num_features = len(X.columns)

## Define and compile the model

In [8]:
model = Sequential()
model.add(InputLayer(input_shape=(num_features,)))
model.add(Dense(20, activation= 'relu' ))
model.add(Dropout(0.2))
model.add(Dense(10, activation= 'relu' ))
model.add(Dense(10, activation= 'relu' ))
model.add(Dense(num_classes, activation= 'softmax' ))
model.compile(loss= 'categorical_crossentropy' , optimizer= 'adam' , metrics=[ 'accuracy' ])

## Fit the model to our data

In [9]:
model.fit(X_train, y_train,
          epochs=20,
          batch_size= 10)
score = model.evaluate(X_test, y_test, batch_size=10)

Epoch 1/20
1227/1227 [==============================] - 2s 2ms/step - loss: 1.3607 - accuracy: 0.4757
Epoch 2/20
1227/1227 [==============================] - 2s 1ms/step - loss: 0.6636 - accuracy: 0.7452
Epoch 3/20
1227/1227 [==============================] - 2s 1ms/step - loss: 0.4580 - accuracy: 0.8341
Epoch 4/20
1227/1227 [==============================] - 2s 1ms/step - loss: 0.3638 - accuracy: 0.8658
Epoch 5/20
1227/1227 [==============================] - 2s 2ms/step - loss: 0.3064 - accuracy: 0.8888
Epoch 6/20
1227/1227 [==============================] - 2s 2ms/step - loss: 0.2611 - accuracy: 0.9056
Epoch 7/20
1227/1227 [==============================] - 3s 2ms/step - loss: 0.2114 - accuracy: 0.9253
Epoch 8/20
1227/1227 [==============================] - 2s 2ms/step - loss: 0.1840 - accuracy: 0.9380
Epoch 9/20
1227/1227 [==============================] - 2s 2ms/step - loss: 0.1563 - accuracy: 0.9451
Epoch 10/20
1227/1227 [==============================] - 2s 2ms/step - loss: 0.141

In [12]:
import mediapipe as mp
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import string

In [14]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

update_time = 10
save_time = 500

In [15]:
def flatten_landmarks(hand_landmarks):
    keypoints = []
    for data_point in hand_landmarks.landmark:
        keypoints.append(data_point.x)
        keypoints.append(data_point.y)
        keypoints.append(data_point.z)
    return keypoints
    
def transform_array(array):
    return list(np.array(array) * (1 / max(array)))

In [24]:
# For webcam input:
cap = cv2.VideoCapture(1)
with mp_hands.Hands(
    max_num_hands=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:

  i = 0
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # Flip the image horizontally for a later selfie-view display, and convert
    # the BGR image to RGB.
    image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    results = hands.process(image)

    key = cv2.waitKey(update_time)
    i = i + update_time
    # Draw the hand annotations on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    if results.multi_hand_landmarks:
        hand_landmarks = results.multi_hand_landmarks[0]
        if i >= save_time:
          i = 0
          flat_keypoints = flatten_landmarks(hand_landmarks)
          flat_keypoints = transform_array(flat_keypoints)
          
          prediction = model.predict([flat_keypoints])
          prediction_label = classes[np.argmax(prediction)]
          print(f'Predicted letter: {prediction_label}')
          
        mp_drawing.draw_landmarks(
            image,
            hand_landmarks,
            mp_hands.HAND_CONNECTIONS,
            mp_drawing_styles.get_default_hand_landmarks_style(),
            mp_drawing_styles.get_default_hand_connections_style())
            
    if key == ord(' '):
      break
            

    cv2.imshow('MediaPipe Hands', image)
    
cap.release()

Predicted letter: d
Predicted letter: g
Predicted letter: g
Predicted letter: g
